## Assignment 1: Part-of-Speech Tagging Using Hidden Markov Model (HMM)

## Objective:
Implemented a Part-of-speech tagging system using a Hidden Markov Model (HMM). The datasets contain sentences where each word is paired with its corresponding PoS tag. The goal is to correctly tag words in sentences by leveraging the Viterbi algorithm.


# Loading Training Corpus

In [ ]:
import pandas as pd

file_path = "https://raw.githubusercontent.com/girishofficial/Viterbi_algorithm/refs/heads/main/TRAIN.csv"
df = pd.read_csv(file_path,names=["sen"])

In [ ]:
df

,sen
0,"[('03/01/2001', 'NUM'), ('01:35', 'NUM'), ('PM..."
1,"[('The', 'DET'), ('industry', 'NOUN'), ('has',..."
2,"[('Love', 'VERB'), ('this', 'DET'), ('place', ..."
3,"[('This', 'DET'), ('problem', 'NOUN'), ('of', ..."
4,"[('Esso', 'PROPN'), ('said', 'VERB'), ('0', 'X..."
...,...
52494,"[('Superstition', 'NOUN'), ('has', 'VERB'), ('..."
52495,"[('You', 'PRON'), ('now', 'ADV'), ('should', '..."
52496,"[('But', 'CONJ'), ('in', 'ADP'), ('the', 'DET'..."
52497,"[('Specifically', 'ADV'), (',', 'PUNCT'), ('Je..."


In [ ]:
df["sen"][0]
type(df['sen'][0][1])

str

In [ ]:
import ast

df["sen"] = df["sen"].apply(ast.literal_eval)
print(type(df["sen"][0]))

<class 'list'>


In [ ]:
df['sen'][0]

[('03/01/2001', 'NUM'), ('01:35', 'NUM'), ('PM', 'NOUN')]

# **Pre-Processing the Dataset**

**Split the data into words and their respective tags for processing**

In [ ]:
import pandas as pd
import ast

# column_name = df.columns[0]
column_name = 'sen'

pairs = []
for tup in df[column_name]:
    for pair in tup:
        pairs.append(pair)

df_expanded = pd.DataFrame(pairs, columns=["word", "tag"])

In [ ]:
df_expanded.head()

,word,tag
0,03/01/2001,NUM
1,01:35,NUM
2,PM,NOUN
3,The,DET
4,industry,NOUN


**Finding unique tags and words in our dataset**

In [ ]:
uniqueTags = df_expanded["tag"].unique()
print("Unique Tags in Dataset:", len(uniqueTags))
uniqueWords = df_expanded["word"].unique();
print("Unique Words in Dataset:", len(uniqueWords))

Unique Tags in Dataset: 21
Unique Words in Dataset: 58110


# Estimate HMM Parameters using the training set

**Finding Initial Probability**

In [ ]:
from collections import defaultdict

initial_prob = defaultdict(int)
total_sentences = len(df)

for i in df["sen"]:
    tag = i[0][1]
    initial_prob[tag] += 1

for tag in initial_prob:
    initial_prob[tag] = initial_prob[tag] / total_sentences

print("----------------------------------------Initial Probabilities:----------------------------------------")
for tag, prob in initial_prob.items():
    print(f"{tag} : {prob:}")

----------------------------------------Initial Probabilities:----------------------------------------
NUM : 0.022686146402788626
DET : 0.19897521857559192
VERB : 0.04920093716070782
PROPN : 0.04114364083125393
PRON : 0.16137450237147374
NOUN : 0.1239833139678851
ADP : 0.11245928493876074
AUX : 0.006533457780148193
. : 0.04421036591173165
PRT : 0.023657593477971008
CCONJ : 0.005409626850035239
INTJ : 0.008533495876111926
ADV : 0.08525876683365398
SCONJ : 0.008666831749176174
CONJ : 0.03701022876626221
_ : 0.01055258195394198
PUNCT : 0.009695422769957524
X : 0.005962018324158555
ADJ : 0.04163888835977828
SYM : 0.0017143183679689136
PART : 0.0013333587306424883


**Calculation Of Transition and Emission Probability**

In [ ]:
tag_counts = {}
transition_counts = {}
word_counts = {}

prev_tag = None

for _, row in df_expanded.iterrows():
    word = row["word"]
    tag = row["tag"]

    if tag in tag_counts:
        tag_counts[tag] += 1
    else:
        tag_counts[tag] = 1

    if prev_tag is not None:=
        if prev_tag not in transition_counts:
            transition_counts[prev_tag] = {}
        if tag in transition_counts[prev_tag]:
            transition_counts[prev_tag][tag] += 1
        else:
            transition_counts[prev_tag][tag] = 1
    prev_tag = tag

for _, row in df_expanded.iterrows():
    word = row["word"]
    tag = row["tag"]

    if tag not in word_counts:
        word_counts[tag] = {}
    if word in word_counts[tag]:
        word_counts[tag][word] += 1
    else:
        word_counts[tag][word] = 1

In [ ]:
print("Tag Counts:", tag_counts)
print("Transition Counts:", transition_counts)
print("Word Counts:", word_counts)


Tag Counts: {'NUM': 17148, 'NOUN': 240200, 'DET': 115786, 'VERB': 151833, 'ADJ': 76692, '.': 94933, 'PUNCT': 23826, 'ADP': 125188, 'CONJ': 27234, 'PROPN': 19556, 'X': 13180, 'PRON': 48875, 'ADV': 49059, 'AUX': 12771, 'INTJ': 755, 'CCONJ': 6535, 'PRT': 18688, 'PART': 7586, 'SCONJ': 3720, '_': 2668, 'SYM': 739}
Transition Counts: {'NUM': {'NUM': 918, 'NOUN': 6198, 'VERB': 610, 'ADJ': 862, '.': 3167, 'PROPN': 196, 'AUX': 44, 'X': 652, 'ADP': 1871, 'DET': 210, 'PUNCT': 1118, 'SYM': 144, 'CONJ': 459, 'PRON': 157, 'ADV': 265, 'PRT': 70, 'PART': 60, 'CCONJ': 112, '_': 12, 'SCONJ': 15, 'INTJ': 8}, 'NOUN': {'DET': 3774, 'VERB': 33904, '.': 53525, 'PUNCT': 10009, 'ADP': 58751, 'CONJ': 12096, 'NOUN': 36310, 'PROPN': 445, 'PRT': 3421, 'NUM': 2050, 'PART': 1425, 'CCONJ': 2402, 'AUX': 2451, 'X': 3668, 'ADJ': 3169, 'PRON': 5242, 'ADV': 6438, 'SCONJ': 790, '_': 174, 'SYM': 125, 'INTJ': 31}, 'DET': {'NOUN': 70465, 'ADJ': 28003, 'PROPN': 1962, 'VERB': 6718, 'NUM': 1325, 'DET': 760, 'PUNCT': 195, 'ADP': 

In [ ]:
transition_prob = {}
for t1 in transition_counts:
    transition_prob[t1] = {}
    for t2 in transition_counts[t1]:
        transition_prob[t1][t2] = transition_counts[t1][t2] / tag_counts[t1]

In [ ]:
print("\n---------------------------------------- Transition Probabilities (Tag → Next Tag) -------------------------------------------")
for t1 in transition_prob:
    for t2 in transition_prob[t1]:
        print(f"{t1} → {t2} : {transition_prob[t1][t2]}")


---------------------------------------- Transition Probabilities (Tag → Next Tag) -------------------------------------------
NUM → NUM : 0.053533939818054585
NUM → NOUN : 0.3614415675297411
NUM → VERB : 0.035572661534872874
NUM → ADJ : 0.05026825285747609
NUM → . : 0.18468626078843015
NUM → PROPN : 0.011429904362024726
NUM → AUX : 0.0025658968975973873
NUM → X : 0.03802192675530674
NUM → ADP : 0.10910893398647073
NUM → DET : 0.01224632610216935
NUM → PUNCT : 0.06519710753440634
NUM → SYM : 0.008397480755773267
NUM → CONJ : 0.026766969909027293
NUM → PRON : 0.009155586657336132
NUM → ADV : 0.015453697224166084
NUM → PRT : 0.004082108700723117
NUM → PART : 0.0034989503149055285
NUM → CCONJ : 0.006531373921156987
NUM → _ : 0.0006997900629811056
NUM → SCONJ : 0.0008747375787263821
NUM → INTJ : 0.00046652670865407047
NOUN → DET : 0.015711906744379684
NOUN → VERB : 0.14114904246461282
NOUN → . : 0.22283513738551208
NOUN → PUNCT : 0.04166944213155704
NOUN → ADP : 0.24459200666111575
NOUN →

In [ ]:
emission_prob = {}
for tag in word_counts:
    emission_prob[tag] = {}
    for word in word_counts[tag]:
        emission_prob[tag][word] = word_counts[tag][word] / tag_counts[tag]

In [ ]:
print("\n----------------------------------------- Emission Probabilities (Tag → Word) ------------------------------------------------")
for tag in emission_prob:
    print(f"\n{tag}:")
    for word, prob in sorted(emission_prob[tag].items(), key=lambda x: -x[1])[:5]:  # Show top 10 words
        print(f"  {word}: {prob}")


----------------------------------------- Emission Probabilities (Tag → Word) ------------------------------------------------

NUM:
  one: 0.11913925822253324
  two: 0.06601352927455097
  1: 0.03318171215302076
  2: 0.03084907860975041
  three: 0.02775833916491719

NOUN:
  time: 0.006065778517901748
  years: 0.003671940049958368
  people: 0.003459616985845129
  Af: 0.0034429641965029144
  year: 0.0032639467110741048

DET:
  the: 0.4709982208557166
  a: 0.16907052666125438
  The: 0.05590485896395073
  his: 0.03307826507522498
  this: 0.03078092342770283

VERB:
  is: 0.05399353236779883
  was: 0.04057089038614794
  be: 0.031640025554392
  have: 0.02413177635955293
  are: 0.024079086891518973

ADJ:
  other: 0.01927189276586867
  new: 0.01418661659625515
  more: 0.011996036092421634
  many: 0.011057215876492985
  such: 0.010744275804516768

.:
  ,: 0.4092359874859111
  .: 0.33306647846375864
  ``: 0.05227897569865063
  '': 0.05188922713914024
  ;: 0.04116587488017865

PUNCT:
  .: 0.36716

In [ ]:
# print("Sample Transition Probabilities:", list(transition_prob.items())[:5])
# print("Sample Emission Probabilities:", list(emission_prob.items())[:5])

In [ ]:
print("Transition Probability Example:", transition_prob.get("NOUN", "Not Available"))
print("Emission Probability Example:", emission_prob.get("NOUN", "Not Available"))

Transition Probability Example: {'DET': 0.015711906744379684, 'VERB': 0.14114904246461282, '.': 0.22283513738551208, 'PUNCT': 0.04166944213155704, 'ADP': 0.24459200666111575, 'CONJ': 0.05035803497085762, 'NOUN': 0.15116569525395504, 'PROPN': 0.0018526228143213989, 'PRT': 0.014242298084929226, 'NUM': 0.008534554537885096, 'PART': 0.00593255620316403, 'CCONJ': 0.01, 'AUX': 0.010203996669442132, 'X': 0.015270607826810991, 'ADJ': 0.013193172356369692, 'PRON': 0.02182348043297252, 'ADV': 0.026802664446294755, 'SCONJ': 0.003288925895087427, '_': 0.0007243963363863448, 'SYM': 0.0005203996669442131, 'INTJ': 0.00012905911740216487}
Emission Probability Example: {'PM': 0.00046627810158201497, 'industry': 0.0007160699417152373, 'year': 0.0032639467110741048, 'place': 0.0022356369691923397, 'problem': 0.0013863447127393839, 'balance': 0.0003372189841798501, 'numbers': 0.0005120732722731058, 'generalists': 8.32639467110741e-06, 'specialists': 9.159034138218151e-05, 'network': 0.00013322231473771856

# Viterbi Algorithm Implementation

In [ ]:
def initialize_viterbi(sentence, states, initial_prob, emission_prob):
    Viterbi = []
    Backpointer = []

    for i in range(len(states)):
        state = states[i]
        if state in initial_prob:
            init_prob = initial_prob[state]
        else:
            init_prob = 1e-6

        if state in emission_prob and sentence[0] in emission_prob[state]:
            emit_prob = emission_prob[state][sentence[0]]
        else:
            emit_prob = 1e-6

        Viterbi.append([init_prob * emit_prob])
        Backpointer.append([0])

    return Viterbi, Backpointer

def fill_viterbi(sentence, states, Viterbi, Backpointer, transition_prob, emission_prob):
    for t in range(1, len(sentence)):
        for s in range(len(states)):
            max_prob = 0
            best_prev_state = 0
            for k in range(len(states)):
                if states[k] in transition_prob and states[s] in transition_prob[states[k]]:
                    trans_prob = transition_prob[states[k]][states[s]]
                else:
                    trans_prob = 1e-6

                if states[s] in emission_prob and sentence[t] in emission_prob[states[s]]:
                    emit_prob = emission_prob[states[s]][sentence[t]]
                else:
                    emit_prob = 1e-6

                prob = Viterbi[k][t - 1] * trans_prob * emit_prob
                if prob > max_prob:
                    max_prob = prob
                    best_prev_state = k

            Viterbi[s].append(max_prob)
            Backpointer[s].append(best_prev_state)

    return Viterbi, Backpointer

def backtrack_viterbi(Viterbi, Backpointer, states):
    T = len(Viterbi[0])
    max_value = 0
    last_state = 0
    for i in range(len(Viterbi)):
        if Viterbi[i][T - 1] > max_value:
            max_value = Viterbi[i][T - 1]
            last_state = i

    best_sequence = [last_state]

    for t in range(T - 1, 0, -1):
        last_state = Backpointer[last_state][t]
        best_sequence.append(last_state)

    best_sequence.reverse()
    predicted_tags = []
    for i in best_sequence:
        predicted_tags.append(states[i])

    return predicted_tags

def viterbi_algorithm(sentence, states, initial_prob, transition_prob, emission_prob):
    Viterbi, Backpointer = initialize_viterbi(sentence, states, initial_prob, emission_prob)
    Viterbi, Backpointer = fill_viterbi(sentence, states, Viterbi, Backpointer, transition_prob, emission_prob)
    return backtrack_viterbi(Viterbi, Backpointer, states)


# Model Evaluation

In [ ]:
import pandas as pd

file_path = "https://raw.githubusercontent.com/girishofficial/Viterbi_algorithm/refs/heads/main/TEST.csv"
df_test = pd.read_csv(file_path,names=["sen"])

print(df_test.head())


                                                 sen
0                                  [('DF', 'PROPN')]
1  [('03/01/2001', 'NUM'), ('01:35', 'NUM'), ('PM...
2  [('The', 'DET'), ('industry', 'NOUN'), ('has',...
3  [('Love', 'VERB'), ('this', 'DET'), ('place', ...
4  [('This', 'DET'), ('problem', 'NOUN'), ('of', ...


In [ ]:
df_test["sen"] = df_test["sen"].apply(ast.literal_eval)
print(type(df_test["sen"][0]))

<class 'list'>


In [ ]:
word_sequences = []

for sentence in df_test["sen"]:
    words = []
    for word, tag in sentence:
        words.append(word)
    word_sequences.append(words)
print(word_sequences[3])

['Love', 'this', 'place', '!!']


In [ ]:
states = df_expanded["tag"].unique()

predicted_tags_list = []

for words in word_sequences:
    predicted_tags = viterbi_algorithm(words, states, initial_prob, transition_prob, emission_prob)
    predicted_tags_list.append(predicted_tags)

In [ ]:
true_tags_list = []

for sentence in df_test["sen"]:
    tags_only = []
    for word, tag in sentence:
        tags_only.append(tag)
    true_tags_list.append(tags_only)

In [ ]:
total_words = 0
correct_predictions = 0

for i in range(len(true_tags_list)):
    for j in range(len(true_tags_list[i])):
        total_words += 1
        if true_tags_list[i][j] == predicted_tags_list[i][j]:
            correct_predictions += 1

accuracy = (correct_predictions / total_words) * 100
print(f"Viterbi Algorithm Accuracy: {accuracy:.2f}%")

Viterbi Algorithm Accuracy: 88.99%


## confusion matrix on test data

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

all_tags = sorted(set(tag for tags in true_tags_list for tag in tags))

conf_matrix = confusion_matrix(
    [tag for tags in true_tags_list for tag in tags],
    [tag for tags in predicted_tags_list for tag in tags],
    labels=all_tags
)

plt.figure(figsize=(20, 5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=all_tags, yticklabels=all_tags)

plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.title("Confusion Matrix")
plt.show()

# **Effect of Lemmatization**

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize_word(word):
    return lemmatizer.lemmatize(word)

Lemmatization on train data

In [ ]:
df_expanded["word"] = df_expanded["word"].apply(lemmatize_word)

Lemmatization on test data

In [ ]:
word_sequences = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in word_sequences]

# **Viterbi algorithm on lemmatized data**

In [ ]:
states = df_expanded["tag"].unique()
predicted_tags_list = []

for words in word_sequences:
    predicted_tags = viterbi_algorithm(words, states, initial_prob, transition_prob, emission_prob)
    predicted_tags_list.append(predicted_tags)  # Store predicted tags

In [ ]:
predicted_tags_list

[['PROPN'],
 ['NUM', 'NUM', 'NOUN'],
 ['DET', 'NOUN', 'PRT', 'VERB', 'NUM', 'ADP', 'DET', 'ADJ', 'NOUN', '.'],
 ['VERB', 'DET', 'NOUN', 'PUNCT'],
 ['DET',
  'NOUN',
  'ADP',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'NOUN',
  'CONJ',
  'NOUN',
  'VERB',
  'VERB',
  'VERB',
  'ADP',
  'DET',
  'ADJ',
  'NOUN',
  'NOUN',
  '.'],
 ['PROPN', 'VERB', 'X', 'DET', 'NOUN', 'NOUN', 'VERB', 'NOUN', 'NOUN', '.'],
 ['PRON', 'VERB', 'DET', 'ADJ', 'NOUN', '.'],
 ['NOUN', '.', '.'],
 ['DET',
  'NOUN',
  'PRT',
  'DET',
  'NOUN',
  'ADP',
  'NOUN',
  'ADP',
  'DET',
  'ADJ',
  '.'],
 ['ADP',
  'DET',
  'NOUN',
  'CONJ',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'ADJ',
  'NOUN',
  'CONJ',
  'ADJ',
  'NOUN',
  'DET',
  'NOUN',
  '.',
  'ADV',
  'VERB',
  'ADP',
  'DET',
  'NOUN',
  'ADP',
  'DET',
  'NOUN',
  '.',
  'ADP',
  'NUM',
  'ADJ',
  'NOUN',
  '.',
  '.'],
 ['AUX',
  'PRON',
  'INTJ',
  'VERB',
  'DET',
  'ADJ',
  'NOUN',
  'ADP',
  'DET',
  'NO

In [ ]:
true_tags_list = []

for sentence in df_test["sen"]:
    tags_only = []
    for word, tag in sentence:
        tags_only.append(tag)
    true_tags_list.append(tags_only)

In [ ]:
total_words = 0
correct_predictions = 0

for i in range(len(true_tags_list)):
    for j in range(len(true_tags_list[i])):
        total_words += 1
        if true_tags_list[i][j] == predicted_tags_list[i][j]:
            correct_predictions += 1

accuracy = (correct_predictions / total_words) * 100
print(f"Viterbi Algorithm Accuracy: {accuracy:.2f}%")

Viterbi Algorithm Accuracy: 87.59%


# Summary

### Accuracy without Lemmatization : 88.98%
### Accuracy with Lemmatization : 87.59%